In [ ]:
import torch

from data import data_helper
from data.data_helper import available_datasets
from models import model_factory
from optimizer.optimizer_helper import get_optim_and_scheduler
from torch.nn import functional as F
from torch import nn

from utils.Logger import Logger

import torchvision
import matplotlib.pyplot as plt

from train_jigsaw import Trainer
from utils import vis

class Container():
    pass

args = Container()
args.batch_size = 128
args.n_classes = 7
args.learning_rate = 0.001
args.epochs = 30
args.network = "caffenet"
args.val_size = 0.1
args.tf_logger = True
args.folder_name = "odd_one_out_gridsearch" # odd_one_out
args.jigsaw_n_classes = 31

args.train_all = True
# args.jig_weight = 0.9
# args.bias_whole_image = 0.8
args.TTA = False
args.suffix = ""

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

source = sorted(["photo", "cartoon", "sketch", "art_painting"])
args.classify_only_sane = True


for args.ooo_weight in [0.1, 0.4, 0.7, 1.0]:
    for args.jig_weight in [0.1, 0.4, 0.7, 1.0]: 
        for args.bias_whole_image in [0.2, 0.5, 0.8]:
            for k, x in enumerate(source):
                args.source = source[:k]+source[k+1:]
                args.target = x
                for i in range(3):
                    print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                                      args.target, 
                                                                      args.jigsaw_n_classes,
                                                                      i))
                    trainer = Trainer(args, device)
                    trainer.do_training()


cartoon-photo-sketch to art_painting - 31 jigsaw classes, split 0
Using Caffe AlexNet
Dataset size: train 7150, val 793, test 2048
Step size: 24
Saving to /home/enoon/code/2018/JigsawDA/myJigsaw/utils/../logs/odd_one_out_gridsearch/cartoon-photo-sketch_to_art_painting/eps30_bs128_lr0.001_class7_jigClass31_jigWeight0.1_oooW0.1_TAll_bias0.2_classifyOnlySane_403
New epoch - lr: 0.001
0/55 of epoch 1/30 jigsaw : 3.493, class : 2.059, OOO : 2.818 - acc jigsaw : 4.69, class : 13.28, OOO : 11.72 [bs:128]
30/55 of epoch 1/30 jigsaw : 3.083, class : 0.599, OOO : 2.295 - acc jigsaw : 23.44, class : 53.91, OOO : 21.88 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 78.94, OOO : 97.86
Accuracies on test: jigsaw : 100.00, class : 56.35, OOO : 90.23
New epoch - lr: 0.001
0/55 of epoch 2/30 jigsaw : 2.934, class : 0.603, OOO : 2.116 - acc jigsaw : 25.00, class : 52.34, OOO : 28.12 [bs:128]
30/55 of epoch 2/30 jigsaw : 2.830, class : 0.764, OOO : 2.019 - acc jigsaw : 26.56, class : 42.97, OOO : 

30/55 of epoch 21/30 jigsaw : 2.387, class : 0.063, OOO : 1.780 - acc jigsaw : 39.06, class : 53.12, OOO : 35.94 [bs:128]
Accuracies on val: jigsaw : 98.87, class : 91.93, OOO : 98.61
Accuracies on test: jigsaw : 97.27, class : 63.87, OOO : 95.46
New epoch - lr: 0.001
0/55 of epoch 22/30 jigsaw : 2.407, class : 0.033, OOO : 1.780 - acc jigsaw : 36.72, class : 55.47, OOO : 36.72 [bs:128]
30/55 of epoch 22/30 jigsaw : 2.343, class : 0.129, OOO : 1.765 - acc jigsaw : 36.72, class : 50.00, OOO : 38.28 [bs:128]
Accuracies on val: jigsaw : 99.12, class : 92.31, OOO : 98.74
Accuracies on test: jigsaw : 98.73, class : 60.79, OOO : 97.71
New epoch - lr: 0.001
0/55 of epoch 23/30 jigsaw : 2.351, class : 0.116, OOO : 1.687 - acc jigsaw : 37.50, class : 53.91, OOO : 35.94 [bs:128]
30/55 of epoch 23/30 jigsaw : 2.096, class : 0.093, OOO : 1.508 - acc jigsaw : 40.62, class : 60.94, OOO : 46.88 [bs:128]
Accuracies on val: jigsaw : 98.87, class : 91.30, OOO : 98.61
Accuracies on test: jigsaw : 98.49, 

30/55 of epoch 11/30 jigsaw : 2.705, class : 0.241, OOO : 1.982 - acc jigsaw : 26.56, class : 49.22, OOO : 30.47 [bs:128]
Accuracies on val: jigsaw : 99.62, class : 90.54, OOO : 98.49
Accuracies on test: jigsaw : 98.78, class : 63.33, OOO : 95.90
New epoch - lr: 0.001
0/55 of epoch 12/30 jigsaw : 2.708, class : 0.255, OOO : 1.956 - acc jigsaw : 27.34, class : 54.69, OOO : 31.25 [bs:128]
30/55 of epoch 12/30 jigsaw : 2.620, class : 0.117, OOO : 1.831 - acc jigsaw : 32.03, class : 45.31, OOO : 33.59 [bs:128]
Accuracies on val: jigsaw : 99.62, class : 90.67, OOO : 98.87
Accuracies on test: jigsaw : 98.97, class : 57.42, OOO : 96.88
New epoch - lr: 0.001
0/55 of epoch 13/30 jigsaw : 2.600, class : 0.066, OOO : 1.775 - acc jigsaw : 29.69, class : 56.25, OOO : 35.16 [bs:128]
30/55 of epoch 13/30 jigsaw : 2.679, class : 0.341, OOO : 1.859 - acc jigsaw : 28.12, class : 50.78, OOO : 27.34 [bs:128]
Accuracies on val: jigsaw : 99.62, class : 89.28, OOO : 99.12
Accuracies on test: jigsaw : 99.07, 

30/55 of epoch 1/30 jigsaw : 3.096, class : 0.662, OOO : 2.539 - acc jigsaw : 21.88, class : 47.66, OOO : 18.75 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 80.20, OOO : 94.96
Accuracies on test: jigsaw : 100.00, class : 51.37, OOO : 87.79
New epoch - lr: 0.001
0/55 of epoch 2/30 jigsaw : 2.946, class : 0.411, OOO : 2.148 - acc jigsaw : 25.00, class : 51.56, OOO : 29.69 [bs:128]
30/55 of epoch 2/30 jigsaw : 3.024, class : 0.591, OOO : 2.204 - acc jigsaw : 18.75, class : 51.56, OOO : 22.66 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 83.10, OOO : 98.61
Accuracies on test: jigsaw : 100.00, class : 57.13, OOO : 95.41
New epoch - lr: 0.001
0/55 of epoch 3/30 jigsaw : 3.107, class : 0.906, OOO : 2.108 - acc jigsaw : 16.41, class : 58.59, OOO : 18.75 [bs:128]
30/55 of epoch 3/30 jigsaw : 2.988, class : 0.286, OOO : 2.132 - acc jigsaw : 20.31, class : 53.12, OOO : 24.22 [bs:128]
Accuracies on val: jigsaw : 99.87, class : 83.73, OOO : 97.60
Accuracies on test: jigsaw : 99.61, c

Accuracies on val: jigsaw : 99.75, class : 90.67, OOO : 98.74
Accuracies on test: jigsaw : 98.58, class : 64.36, OOO : 96.97
New epoch - lr: 0.001
0/55 of epoch 23/30 jigsaw : 2.454, class : 0.015, OOO : 1.766 - acc jigsaw : 32.03, class : 59.38, OOO : 35.94 [bs:128]
30/55 of epoch 23/30 jigsaw : 2.356, class : 0.059, OOO : 1.661 - acc jigsaw : 37.50, class : 56.25, OOO : 39.06 [bs:128]
Accuracies on val: jigsaw : 99.75, class : 90.04, OOO : 98.87
Accuracies on test: jigsaw : 99.32, class : 59.81, OOO : 98.34
New epoch - lr: 0.001
0/55 of epoch 24/30 jigsaw : 2.438, class : 0.104, OOO : 1.793 - acc jigsaw : 34.38, class : 50.78, OOO : 39.06 [bs:128]
30/55 of epoch 24/30 jigsaw : 2.638, class : 0.139, OOO : 1.896 - acc jigsaw : 29.69, class : 58.59, OOO : 35.16 [bs:128]
Accuracies on val: jigsaw : 99.12, class : 89.79, OOO : 98.11
Accuracies on test: jigsaw : 98.05, class : 57.03, OOO : 96.00
New epoch - lr: 0.0001
0/55 of epoch 25/30 jigsaw : 2.357, class : 0.045, OOO : 1.544 - acc jig

...

Accuracies on test: jigsaw : 99.67, class : 61.31, OOO : 99.72
New epoch - lr: 0.001
0/42 of epoch 10/30 jigsaw : 0.782, class : 0.090, OOO : 0.590 - acc jigsaw : 79.69, class : 84.38, OOO : 79.69 [bs:128]
30/42 of epoch 10/30 jigsaw : 0.737, class : 0.096, OOO : 0.572 - acc jigsaw : 78.91, class : 92.19, OOO : 83.59 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 92.07, OOO : 99.83
Accuracies on test: jigsaw : 99.36, class : 61.80, OOO : 99.52
New epoch - lr: 0.001
0/42 of epoch 11/30 jigsaw : 0.712, class : 0.076, OOO : 0.496 - acc jigsaw : 80.47, class : 88.28, OOO : 81.25 [bs:128]
30/42 of epoch 11/30 jigsaw : 0.691, class : 0.073, OOO : 0.507 - acc jigsaw : 82.81, class : 88.28, OOO : 83.59 [bs:128]
Accuracies on val: jigsaw : 100.00, class : 91.07, OOO : 100.00
Accuracies on test: jigsaw : 99.11, class : 63.68, OOO : 99.31
New epoch - lr: 0.001
0/42 of epoch 12/30 jigsaw : 0.629, class : 0.155, OOO : 0.509 - acc jigsaw : 82.81, class : 80.47, OOO : 82.81 [bs:128]
30/42 of ep

Accuracies on test: jigsaw : 98.24, class : 64.42, OOO : 99.01
Best val 0.933884, corresponding test 0.604989 - best test: 0.653092
It took 586.45

cartoon-photo-sketch to art_painting - 31 jigsaw classes, split 0
Using Caffe AlexNet
Dataset size: train 7150, val 793, test 2048
Step size: 24
Saving to /home/enoon/code/2018/JigsawDA/myJigsaw/utils/../logs/odd_one_out_gridsearch/cartoon-photo-sketch_to_art_painting/eps30_bs128_lr0.001_class7_jigClass31_jigWeight0.7_oooW0.1_TAll_bias0.2_classifyOnlySane_933
New epoch - lr: 0.001
0/55 of epoch 1/30 jigsaw : 3.532, class : 2.002, OOO : 2.844 - acc jigsaw : 3.91, class : 14.06, OOO : 8.59 [bs:128]
30/55 of epoch 1/30 jigsaw : 2.498, class : 1.090, OOO : 2.173 - acc jigsaw : 33.59, class : 48.44, OOO : 28.12 [bs:128]
Accuracies on val: jigsaw : 98.74, class : 81.08, OOO : 96.60
Accuracies on test: jigsaw : 99.02, class : 61.77, OOO : 92.48
New epoch - lr: 0.001
0/55 of epoch 2/30 jigsaw : 2.560, class : 0.638, OOO : 2.100 - acc jigsaw : 32.81

Accuracies on test: jigsaw : 95.70, class : 62.45, OOO : 97.31
New epoch - lr: 0.001
0/55 of epoch 21/30 jigsaw : 0.609, class : 0.389, OOO : 1.533 - acc jigsaw : 85.16, class : 53.12, OOO : 45.31 [bs:128]
30/55 of epoch 21/30 jigsaw : 0.629, class : 0.553, OOO : 1.550 - acc jigsaw : 82.03, class : 47.66, OOO : 41.41 [bs:128]
Accuracies on val: jigsaw : 99.50, class : 89.91, OOO : 99.75
Accuracies on test: jigsaw : 99.41, class : 63.96, OOO : 99.56
New epoch - lr: 0.001
0/55 of epoch 22/30 jigsaw : 0.624, class : 0.156, OOO : 1.495 - acc jigsaw : 80.47, class : 58.59, OOO : 50.00 [bs:128]
30/55 of epoch 22/30 jigsaw : 0.700, class : 0.040, OOO : 1.462 - acc jigsaw : 80.47, class : 49.22, OOO : 46.09 [bs:128]
Accuracies on val: jigsaw : 98.87, class : 90.42, OOO : 99.50
Accuracies on test: jigsaw : 98.19, class : 63.13, OOO : 98.58
New epoch - lr: 0.001
0/55 of epoch 23/30 jigsaw : 0.548, class : 0.055, OOO : 1.456 - acc jigsaw : 83.59, class : 49.22, OOO : 46.88 [bs:128]
30/55 of epoch

Accuracies on test: jigsaw : 97.27, class : 62.26, OOO : 97.56
New epoch - lr: 0.001
0/55 of epoch 11/30 jigsaw : 1.096, class : 0.030, OOO : 1.832 - acc jigsaw : 64.84, class : 53.12, OOO : 39.06 [bs:128]
30/55 of epoch 11/30 jigsaw : 1.146, class : 0.208, OOO : 1.890 - acc jigsaw : 67.97, class : 42.97, OOO : 34.38 [bs:128]
Accuracies on val: jigsaw : 99.12, class : 91.93, OOO : 99.62
Accuracies on test: jigsaw : 97.75, class : 59.42, OOO : 98.34
New epoch - lr: 0.001
0/55 of epoch 12/30 jigsaw : 1.208, class : 0.080, OOO : 2.048 - acc jigsaw : 61.72, class : 42.97, OOO : 28.12 [bs:128]
30/55 of epoch 12/30 jigsaw : 1.074, class : 0.131, OOO : 1.654 - acc jigsaw : 71.09, class : 46.88, OOO : 42.19 [bs:128]
Accuracies on val: jigsaw : 98.49, class : 91.05, OOO : 99.24
Accuracies on test: jigsaw : 95.95, class : 63.67, OOO : 96.19
New epoch - lr: 0.001
0/55 of epoch 13/30 jigsaw : 0.969, class : 0.110, OOO : 1.825 - acc jigsaw : 71.09, class : 41.41, OOO : 39.06 [bs:128]
30/55 of epoch

0/55 of epoch 1/30 jigsaw : 3.493, class : 2.000, OOO : 2.797 - acc jigsaw : 0.78, class : 11.72, OOO : 14.06 [bs:128]
30/55 of epoch 1/30 jigsaw : 2.955, class : 1.269, OOO : 2.407 - acc jigsaw : 20.31, class : 35.16, OOO : 19.53 [bs:128]
Accuracies on val: jigsaw : 99.50, class : 82.09, OOO : 93.44
Accuracies on test: jigsaw : 98.49, class : 60.89, OOO : 81.98
New epoch - lr: 0.001
0/55 of epoch 2/30 jigsaw : 2.590, class : 0.585, OOO : 2.100 - acc jigsaw : 30.47, class : 55.47, OOO : 28.91 [bs:128]
30/55 of epoch 2/30 jigsaw : 2.571, class : 0.675, OOO : 2.107 - acc jigsaw : 28.91, class : 46.88, OOO : 32.81 [bs:128]
Accuracies on val: jigsaw : 98.61, class : 83.35, OOO : 97.10
Accuracies on test: jigsaw : 96.14, class : 58.01, OOO : 90.48
New epoch - lr: 0.001
0/55 of epoch 3/30 jigsaw : 2.618, class : 0.446, OOO : 2.025 - acc jigsaw : 28.91, class : 50.78, OOO : 31.25 [bs:128]
30/55 of epoch 3/30 jigsaw : 2.480, class : 0.455, OOO : 2.138 - acc jigsaw : 32.81, class : 52.34, OOO :

In [ ]:
source = sorted(["photo", "cartoon", "sketch", "art_painting"])
for args.jig_weight in [0.0]:
    print("\n======================\n%g\n===================" % args.jig_weight)
    for args.bias_whole_image in [1.0]: # [None, 0.01, 0.05, 0.1, 0.3, 0.5]:
        for k, x in enumerate(source):
            args.source = source[:k]+source[k+1:]
            args.target = x
            for i in range(3):
                print("\n%s to %s - %d jigsaw classes, split %d" % ("-".join(args.source), 
                                                                  args.target, 
                                                                  args.jigsaw_n_classes,
                                                                  i))
                trainer = Trainer(args, device)
                trainer.do_training()

In [ ]:
trainer = Trainer(args, device)
logger, model = trainer.do_training()

In [ ]:
%matplotlib notebook
print(100*(logger.val_acc["class"][-1] + logger.val_acc["class"][-2])/2.)
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % ("-".join(args.source),
                                                            args.target,args.epochs, args.jig_weight))

In [ ]:
dataloaders = {"train": data_helper.get_dataloader(args.source, args.jigsaw_n_classes, "train"),
           "val": data_helper.get_dataloader(args.target, args.jigsaw_n_classes, "val")}
dataset_sizes = {"train": len(dataloaders["train"].dataset),
                "val": len(dataloaders["val"].dataset)}
print(dataset_sizes)

In [ ]:
def get_optim_and_scheduler(network, epochs, lr):
    from torch import optim
    optimizer = optim.SGD(network.get_params(lr), weight_decay=.0005, momentum=.9, nesterov=True, lr=lr)
    step_size = int(epochs * .8)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size)
    print(step_size)
    return optimizer, scheduler

model_ft = model_factory.get_network("caffenet")(jigsaw_classes=args.jigsaw_n_classes+1, classes=args.n_classes)
model_ft = model_ft.to(device)
# print(model_ft)

optimizer, scheduler = get_optim_and_scheduler(model_ft, args.epochs, args.learning_rate)

In [ ]:
def do_epoch(model, source, target, optimizer, logger, device):
    criterion = nn.CrossEntropyLoss()
    model.train()
    for it, ((data, jig_l, class_l), d_idx) in enumerate(source):
        data, jig_l, class_l = data.to(device), jig_l.to(device), class_l.to(device)
        
        optimizer.zero_grad()
        
        jigsaw_logit, class_logit = model(data)
        jigsaw_loss = criterion(jigsaw_logit, jig_l)
        class_loss = criterion(class_logit[d_idx!=target_id], class_l[d_idx!=target_id])
        _, cls_pred = class_logit.max(dim=1)
        _, jig_pred = jigsaw_logit.max(dim=1)
        loss = class_loss + jigsaw_loss * jig_weight
        
        loss.backward()
        optimizer.step()

        logger.log(it, len(source), {"jigsaw": jigsaw_loss.item(), "class": class_loss.item()},
                  {"jigsaw": torch.sum(jig_pred == jig_l.data).item(), "class":torch.sum(cls_pred == class_l.data).item()},
                  data.shape[0])
        del loss, class_loss, jigsaw_loss, jigsaw_logit, class_logit

    model.eval()
    with torch.no_grad():
        jigsaw_correct = 0
        class_correct = 0
        total = 0
        for it, ((data, jig_l, class_l), d_idx) in enumerate(target):
            data, jig_l, class_l = data.to(device), jig_l.to(device), class_l.to(device)
            jigsaw_logit, class_logit = model(data)
            _, cls_pred = class_logit.max(dim=1)
            _, jig_pred = jigsaw_logit.max(dim=1)
            class_correct += torch.sum(cls_pred == class_l.data)
            jigsaw_correct += torch.sum(jig_pred == jig_l.data)
            total += data.shape[0]
        logger.log_test({"jigsaw": float(jigsaw_correct) / total,
                         "class": float(class_correct) / total})


def do_training(args, model, source, target, optimizer, scheduler, device):
    logger = Logger(args)
    for k in range(args.epochs):
        scheduler.step()
        logger.new_epoch(scheduler.get_lr())
        do_epoch(model, source, target, optimizer, logger, device)
    return logger, model

In [ ]:
jig_weight = args.jig_weight
logger, model = do_training(args, model_ft, dataloaders["train"], dataloaders["val"], optimizer, scheduler, device)

In [ ]:
%matplotlib notebook
print(100*(logger.val_acc["class"][-1] + logger.val_acc["class"][-2])/2.)
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % (str(args.source),args.target,args.epochs, jig_weight))

In [ ]:
def to_plt(inp):
    import numpy as np
    inp = inp.numpy().transpose((1, 2, 0))
    inp = np.clip(inp, 0, 1)
    return inp

conv1 = model_ft.features[0] # models.alexnet(pretrained=True).features[0] #model_ft.features[0]
tmp = conv1.weight.cpu().data
tmp = torchvision.utils.make_grid(tmp,normalize=True)
plt.imshow(to_plt(tmp))
plt.show()

In [ ]:
vis.view_training(logger, "%s->%s eps:%d jigweight:%.1f" % (str(source),target,epochs, jig_weight))

In [ ]:
fig, ax1 = plt.subplots()
for k,v in logger.losses.items():
    ax1.plot(v, label=k)
    l = len(v)
updates = l / len(logger.val_acc)
print(updates)
plt.legend()
ax2 = ax1.twinx()
ax2.plot(range(0,l,int(updates)), logger.val_acc, label="Test acc", c='g')
plt.legend()
plt.show()

In [ ]:
for e,k in enumerate(range(0,l,int(updates))):
    print(k, logger.val_acc[e])

In [ ]:



# iter_c = iter(train_datasets)

# for x in range(5):
#     tmp = next(iter_c)
#     image = to_plt(tmp[0])
#     plt.imshow(image)
#     plt.show()

In [ ]:
from data.data_helper import get_val_dataloader, get_train_dataloader
from os.path import join, dirname
# from data.JigsawLoader import JigsawTestDataset
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision

In [ ]:
# loader = get_val_dataloader("photo",31,batch_size=10,multi=True)
loader, _ = get_train_dataloader(["photo"],31)

In [ ]:
def to_plt(inp):
    inp = inp.numpy().transpose((1, 2, 0))
    return inp

# dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)
# test = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True, drop_last=False)
iter_c = iter(loader)
(tmp, v, c, ooo), d = next(iter_c)
print(tmp[:,0].shape)
for x in range(10):
#     image = tmp[0, x]
    image = torchvision.utils.make_grid(tmp[x],1,normalize=True)
    plt.imshow(to_plt(image))
    plt.show()
    print(ooo[x])

# print(tmp[:,0].shape)
# for x in range(tmp.shape[1]):
# #     image = tmp[0, x]
#     image = torchvision.utils.make_grid(tmp[0, x],1,normalize=True)
#     plt.imshow(to_plt(image))
#     plt.show()
#     print(v[0,x])

    
# print(v.max(), v.min())

In [ ]:
res = []
for k in range(10):
    res.append(torch.rand(5, 100))
res = torch.stack(res,0)
res.mean(1).shape

In [ ]:
from data.JigsawLoader import JigsawDataset
from PIL import Image
import torchvision.transforms as transforms

class JigsawTestDataset(JigsawDataset):
    def __init__(self, *args, **xargs):
        super().__init__(*args, **xargs)
        self._augment_tile = transforms.Compose([
#             transforms.RandomCrop(64),
            transforms.Resize((75, 75), Image.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
    def __getitem__(self, index):
        framename = self.data_path + '/' + self.names[index]
        img = Image.open(framename).convert('RGB')
        img = self._image_transformer(img)

        w = float(img.size[0]) / self.grid_size
        n_grids = self.grid_size ** 2
        tiles = [None] * n_grids
        for n in range(n_grids):
            y = int(n / self.grid_size)
            x = n % self.grid_size
            tile = img.crop([x * w, y * w, (x + 1) * w, (y + 1) * w])
            tile = self._augment_tile(tile)
            tiles[n] = tile

        data = torch.stack(tiles, 0)
        return self.returnFunc(data), 0, int(self.labels[index])


In [ ]:
dataset = JigsawTestDataset("", join('data/txt_lists', 'dslr_train.txt'), patches=False, classes=31)

In [ ]:
import torch
import random

In [ ]:
true = 0
for x in range(100):
    true += 0.1 > random.random()
true